# Master Thesis on the Semantics of (made-up) Names

* Author: Aron Joosse
* Supervisor: Giovanni Cassani
* Institution: Tilburg University

Can take inspiration from: https://github.com/Masetto96/BA-Thesis-form-meaning-mapping/blob/master/form_meaning_mapping.ipynb

# Library Imports

In [1]:
!pip install fasttext --progress-bar off
!pip install -U spacy --progress-bar off
!python -m spacy download en_core_web_sm
import fasttext
import spacy
import numpy as np
import pandas as pd
import re
import pickle

     |████████████████████████████████| 13.9 MB 497 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Data Import

In [2]:
## Being able to access Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True) 

Mounted at /content/drive


In [3]:
## Getting the list of madeup names:

ratings_csv = pd.read_csv("drive/MyDrive/Thesis/Data/giovanni_email_data/avgRatings_annotated.csv",
                          usecols = ["name", "name_type"])

ratings_csv.head(10)

madeup_names = []

for i in ratings_csv.index:                                           ## I can do exactly the same thing for talking & real
  if ratings_csv["name_type"][i] == "madeup":
    madeup_names.append(str(ratings_csv["name"][i]))

madeup_names_lower = list(map(lambda x: x.lower(), madeup_names))

print(madeup_names[:5])
print(len(madeup_names))
print(madeup_names_lower[:5])
print(len(madeup_names_lower))

['Alastor', 'Alecto', 'Amabala', 'Araminta', 'Arcturus']
60
['alastor', 'alecto', 'amabala', 'araminta', 'arcturus']
60


## COCA

In [4]:
path = "drive/My Drive/Thesis/Data/CoCA/Text/"
unclean_path = path + "texts_combined/all_texts_combined.txt"
unclean_corpus = open(unclean_path).read()


In [5]:
print(len(unclean_corpus))
print(unclean_corpus[:100])

2977527143
@@4170367 Headnote # A puzzle has long pervaded the criminal law : why are two offenders who commit 


## Names

# Preprocessing


## Cleaning Corpus

In [6]:
## Loading the English spacy pipeline and removing stopwords

nlp = spacy.load("en_core_web_sm")
nlp.max_length = 10000000000

nlp.Defaults.stop_words.remove('him')
nlp.Defaults.stop_words.remove('her')
nlp.Defaults.stop_words.remove('hers')
nlp.Defaults.stop_words.remove('his')
nlp.Defaults.stop_words.remove('he')
nlp.Defaults.stop_words.remove('she')
nlp.Defaults.stop_words.remove('himself')
nlp.Defaults.stop_words.remove('herself')

In [15]:
def clean_corpus_sentenced(data, corpus_dict, index):
  # Tokenization
  with nlp.select_pipes(disable=["lemmatizer", "tok2vec", "tagger", "parser"]):
    nlp.enable_pipe("senter")
    doc = nlp(data)

  sentence = ""

  for token in doc:
    if token.is_sent_start is True:
      if sentence == "":
        continue
      else:
        corpus_dict[index] = sentence
        sentence = ""
        index += 1
    
    if token.is_upper is True:
      continue
    elif token.is_stop is True:
      continue
    elif str(token).lower() in madeup_names_lower:
      continue
    elif token.is_alpha:
      sentence += str(token).lower()

  return corpus_dict, index

In [ ]:
corpus_dict = {}
index = 0

prev_i = 0

for i in range(2, 2976, 2):
  print(i)
  i *= 1000000
  corpus_dict, index = clean_corpus_sentenced(unclean_corpus[prev_i:i],
                                              corpus_dict,
                                              index)
  prev_i = i

corpus_dict, index = clean_corpus_sentenced(unclean_corpus[prev_i:],
                                            corpus_dict,
                                            index)

print("----------------------")
print(len(corpus_dict), index)

2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220


In [ ]:
pickle_out = open(path + "corpus_dict.pickle", "wb")
pickle.dump(corpus_dict, pickle_out)
pickle_out.close()

drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

In [ ]:
def clean_corpus_unsentenced(data):
    # Tokenization
    with nlp.select_pipes(disable=["lemmatizer", "tok2vec", "tagger", "parser"]):
      nlp.enable_pipe("senter")
      doc = nlp(data)
    print(doc[:150])

    doc_filtered = []

    for token in doc:
      if token.is_upper is True:
        continue
      elif token.is_stop is True:
        continue
      elif str(token).lower() in madeup_names_lower:
        continue
      elif token.is_alpha:
        doc_filtered.append(str(token).lower())
      else: 
        continue

    doc_filtered = " ".join(doc_filtered)

    print(doc_filtered[:500])

    # Remove words with freq < XX

clean_corpus_unsentenced(unclean_corpus)#[:1000000])

## Training fastText and Validating on Word Embeddings Benchmark

In [ ]:
# Skipgram model :
#model = fasttext.train_unsupervised('data.txt', model='skipgram')

#model.save_model("model_filename.bin")

#model = fasttext.load_model("model_filename.bin")

#model.get_nearest_neighbors('asparagus')

#In a similar spirit, one can play around with word analogies. For example, we can see if our model can guess what is to France, and what Berlin is to Germany.
#This can be done with the analogies functionality. It takes a word triplet (like Germany Berlin France) and outputs the analogy:
#model.get_analogies("berlin", "germany", "france")

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
